In [1]:
# =========================
# 1) IMPORT THƯ VIỆN (bám 2.3.2 Bài làm mẫu)
# =========================
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.naive_bayes import MultinomialNB, BernoulliNB

# =========================
# 2) NẠP & SƠ BỘ XỬ LÝ DỮ LIỆU (giống bước 1–2 của bài mẫu)
# =========================
CSV_PATH = "/kaggle/input/mushroom-classification/mushrooms.csv"   # <-- đổi tên file nếu khác
df = pd.read_csv(CSV_PATH)

# Map nhãn 'class' e/p -> 0/1 (edible=0, poisonous=1)
label_col_candidates = [c for c in df.columns if c.lower() in ["class","label","target"]]
assert len(label_col_candidates) >= 1, "Không tìm thấy cột nhãn (class/label/target)"
label_col = label_col_candidates[0]

y = df[label_col].astype(str).str.strip().str.lower().map({"e":0, "edible":0, "p":1, "poisonous":1})
if y.isna().any():
    raise ValueError("Cột nhãn có giá trị ngoài e/p. Hãy kiểm tra dữ liệu.")
y = y.astype(int)

# Tất cả cột còn lại là đặc trưng phân loại
X = df.drop(columns=[label_col]).copy()

# Thay '?' thành 'missing' để xem như một hạng mục hợp lệ
X = X.apply(lambda s: s.astype(str).str.strip().replace({"?":"missing"}))

# =========================
# 3) TÁCH TẬP TRAIN/TEST (80/20) (giống bài mẫu)
# =========================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# =========================
# 4) “VECTOR HÓA” CÁC BIẾN PHÂN LOẠI (tương tự CountVectorizer trong bài mẫu)
#    - OneHotEncoder cho toàn bộ cột (categorical)
#    - Cho phép sparse để dùng tối ưu với Naive Bayes
# =========================
try:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=True)  # sklearn >=1.2
except TypeError:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse=True)         # sklearn <1.2

preprocess = ColumnTransformer(
    transformers=[("cat", ohe, list(X.columns))],
    remainder="drop"
)

# =========================
# 5) HUẤN LUYỆN NAIVE BAYES (giống bước 4 của bài mẫu)
#    - Thử cả MultinomialNB (phù hợp ma trận đếm/one-hot) và BernoulliNB (nhị phân)
# =========================
models = {
    "MultinomialNB": MultinomialNB(alpha=1.0),   # smoothing mặc định
    "BernoulliNB" : BernoulliNB(alpha=1.0)
}

fitted = {}
for name, clf in models.items():
    pipe = Pipeline(steps=[("prep", preprocess), ("nb", clf)])
    pipe.fit(X_train, y_train)
    fitted[name] = pipe

# =========================
# 6) ĐÁNH GIÁ HIỆU QUẢ (bước 5 của bài mẫu)
# =========================
for name, pipe in fitted.items():
    print("="*70)
    print(f"[{name}]")
    y_pred = pipe.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    cm  = confusion_matrix(y_test, y_pred)
    rep = classification_report(y_test, y_pred, digits=4)
    print(f"Accuracy: {acc:.4f}")
    print("Confusion Matrix:\n", cm)
    print("Classification Report:\n", rep)

# =========================
# 7) (TUỲ CHỌN) TỐI ƯU alpha bằng GridSearchCV cho MultinomialNB (minh hoạ 2.3.1)
# =========================
param_grid = {"nb__alpha": [0.1, 0.5, 1.0, 2.0, 3.0]}
grid = GridSearchCV(
    estimator=Pipeline(steps=[("prep", preprocess), ("nb", MultinomialNB())]),
    param_grid=param_grid,
    scoring="accuracy",   # có thể đổi F1-macro nếu cần cân bằng lớp
    cv=5, n_jobs=-1, verbose=1, refit=True
)
grid.fit(X_train, y_train)

print("="*70)
print("[GridSearchCV - MultinomialNB] best params:", grid.best_params_)
print("Best CV accuracy:", grid.best_score_)
y_pred_gs = grid.predict(X_test)
print("Test accuracy (best alpha):", accuracy_score(y_test, y_pred_gs))
print("Classification Report (GridSearch best):\n", classification_report(y_test, y_pred_gs, digits=4))


[MultinomialNB]
Accuracy: 0.9458
Confusion Matrix:
 [[835   7]
 [ 81 702]]
Classification Report:
               precision    recall  f1-score   support

           0     0.9116    0.9917    0.9499       842
           1     0.9901    0.8966    0.9410       783

    accuracy                         0.9458      1625
   macro avg     0.9508    0.9441    0.9455      1625
weighted avg     0.9494    0.9458    0.9456      1625

[BernoulliNB]
Accuracy: 0.9329
Confusion Matrix:
 [[829  13]
 [ 96 687]]
Classification Report:
               precision    recall  f1-score   support

           0     0.8962    0.9846    0.9383       842
           1     0.9814    0.8774    0.9265       783

    accuracy                         0.9329      1625
   macro avg     0.9388    0.9310    0.9324      1625
weighted avg     0.9373    0.9329    0.9326      1625

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[GridSearchCV - MultinomialNB] best params: {'nb__alpha': 0.1}
Best CV accuracy: 0.9769199